In [2]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [4]:
with open("train_MT.words.txt", 'r') as train:
	train_data = train.readlines()
lines_all = []
for data in train_data:
	lines = data.split()
	lines_all.append(lines)
print(len(lines_all))

with open("train_MT.tags.txt", 'r') as tags:
	tag_data = tags.readlines()
tags_all = []
for tag in tag_data:
	tags = tag.split()
	tags_all.append(tags)
print(len(tags_all))

4199
4199


In [5]:
zipped_lists = zip(lines_all, tags_all)

dfs = []
for idx, (line_list, tag_list) in enumerate(zipped_lists):
	temp_df = pd.DataFrame(list(zip(line_list, tag_list)), columns = ["Token", "Tag"])
	temp_df["Sentence #"] = idx
	dfs.append(temp_df)
all_data = pd.concat(dfs)
print(all_data)

          Token Tag  Sentence #
0            he   O           0
1           was   O           0
2      clinical   O           0
3           and   O           0
4      research   O           0
..          ...  ..         ...
14      process   O        4198
15           of   O        4198
16     cellular   O        4198
17  respiration   O        4198
18            .   O        4198

[73429 rows x 3 columns]


In [6]:
train = all_data
train

,Token,Tag,Sentence #
0,he,O,0
1,was,O,0
2,clinical,O,0
3,and,O,0
4,research,O,0
...,...,...,...
14,process,O,4198
15,of,O,4198
16,cellular,O,4198
17,respiration,O,4198


In [7]:
with open("test_MT.words.txt", 'r') as f:
	test_data = f.readlines()
lines_all = []
for data in test_data:
	lines = data.split()
	lines_all.append(lines)
print(len(lines_all))

with open("test_MT.tags.txt", 'r') as f2:
	tag_data = f2.readlines()
tags_all = []
for tag in tag_data:
	tags = tag.split()
	tags_all.append(tags)
print(len(tags_all))

800
800


In [8]:
zipped_lists = zip(lines_all, tags_all)

dfs = []
for idx, (line_list, tag_list) in enumerate(zipped_lists):
	temp_df = pd.DataFrame(list(zip(line_list, tag_list)), columns = ["Token", "Tag"])
	temp_df["Sentence #"] = idx
	dfs.append(temp_df)
all_data_test = pd.concat(dfs)
print(all_data_test)

test = all_data_test

         Token   Tag  Sentence #
0   importance     O           0
1           of     O           0
2        lipid  B-MT           0
3   metabolism  I-MT           0
4      general     O           0
..         ...   ...         ...
29          va     O         799
30    lakeside     O         799
31     medical     O         799
32      center     O         799
33           .     O         799

[14491 rows x 3 columns]


In [9]:
class ProcessSentences(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Token"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

,Token,Tag,Sentence #
0,importance,O,0
1,of,O,0
2,lipid,B-MT,0
3,metabolism,I-MT,0
4,general,O,0
...,...,...,...
29,va,O,799
30,lakeside,O,799
31,medical,O,799
32,center,O,799


In [10]:
sentences_data_train = ProcessSentences(train)
sentences_data_test = ProcessSentences(test)
sentences_train = [[word[0] for word in sentence] for sentence in sentences_data_train.sentences]
sentences_test = [[word[0] for word in sentence] for sentence in sentences_data_test.sentences]

In [11]:
labels_train = [[s[1] for s in sentence] for sentence in sentences_data_train.sentences]
labels_test = [[s[1] for s in sentence] for sentence in sentences_data_test.sentences]
labels_test

[['O', 'O', 'B-MT', 'I-MT', 'O', 'O', 'O', 'B-MT', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MT',
  'I-MT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MT',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MT',
  'I-MT',
  'B-MT',
  'I-MT',
  'B-MT',
  'B-MT',
  'I-MT',
  'I-MT',
  'I-MT',
  'I-MT',
  'I-MT',
  'I-MT',
  'B-MT',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'B-MT', 'I-MT', 'B-MT', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'B-MT',
  'I-MT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MT',
  'B-MT',
  'I-MT',
  'I-MT',
  'B-MT',
  'B-MT',
  'I-MT',
  'I-MT',
  'I-MT',
  'B-MT',
  'B-MT',
  'I-MT',
  'O',
  'B-MT',
  'I-MT',
  'B-MT',
  'O',
  'O',
  'O',
  'B-MT',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'B-MT', 'I-MT', 'O'],
 ['O', 'O', 'B-MT', 'I-MT', 'I-MT', 'B-MT', 'O', 'O', 'B-MT', 'I-MT', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MT',
  'I-MT',
  '

In [12]:
tag_values = list(set(train["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [13]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import transformers
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences

torch.__version__

'1.10.0+cu111'

In [14]:
MAX_LEN = 270
# Batch size
bs = 8 

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
# !nvidia-smi
n_gpu

0

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)

In [17]:
def tokenize_and_keep_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize each word and count number of its subwords
        # We force conversion to string to avoid errors with float elements
        tokenized_word = tokenizer.tokenize(str(word))
        n_subwords = len(tokenized_word)

        # The tokenized word is added to the resulting tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # The same label is added to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [18]:
tokenized_texts_and_labels_train = [
    tokenize_and_keep_labels(sent, labs)
    for sent, labs in zip(sentences_train, labels_train)
]

tokenized_texts_and_labels_test = [
    tokenize_and_keep_labels(sent, labs)
    for sent, labs in zip(sentences_test, labels_test)
]

In [19]:
tokenized_texts_train = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_train]
labels_train = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_train]

tokenized_texts_test = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_test]
labels_test = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_test]

In [20]:
input_ids_train = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_train],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")


tags_train = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_train],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

tags_test = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_test],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [21]:
attention_masks_train = [[float(i != 0.0) for i in ii] for ii in input_ids_train]
attention_masks_test = [[float(i != 0.0) for i in ii] for ii in input_ids_test]

In [22]:
train_inputs = torch.tensor(input_ids_train)
train_tags = torch.tensor(tags_train)
train_masks = torch.tensor(attention_masks_train)

test_inputs = torch.tensor(input_ids_test)
test_tags = torch.tensor(tags_test)
test_masks = torch.tensor(attention_masks_test)

In [23]:
train_data = TensorDataset(train_inputs, train_masks, train_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

test_data = TensorDataset(test_inputs, test_masks, test_tags)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)

In [24]:
from transformers import BertForTokenClassification, AdamW 
model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
# model.cuda();

In [25]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

# Adam optimizer
optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [26]:
# Import a scheduler to reduce the learning rate 
from transformers import get_linear_schedule_with_warmup

# Number of training epochs; the BERT paper uses 4
epochs = 4
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [27]:
!pip install seqeval

In [28]:
import seqeval
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

In [32]:
%%time 
# To measure execution time of this cell

# Train the model for; the BERT paper uses 4
## Store the average loss after each epoch; these values are used to plot the loss.
loss_values, development_loss_values = [], []
epochs = 1
for _ in range(epochs):
    #
    # Training
    #
    # Set the model into training mode
    model.train()
    # Reset the total loss for each epoch
    total_loss = 0
    print('epoch:', _)
    print(len(train_dataloader))
    for step, batch in enumerate(train_dataloader):
        # Transfer batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Remove previous gradients before each backward pass
        model.zero_grad()
        print('forward pass')
        # forward pass
        # This returns the loss (not the model output) since we have input the labels.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # Get the loss
        loss = outputs[0]
        # Backward pass to compute the gradients
        print('about to backward')
        loss.backward()
        # Train loss
        total_loss += loss.item()
        print(total_loss)
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # Update parameters
        optimizer.step()
        print('here')
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))
    

    # Store each loss value for plotting the learning curve afterwards
    loss_values.append(avg_train_loss)

epoch: 0
525
forward pass
about to backward
0.0665954202413559
here
forward pass
about to backward
0.22125470638275146
here
forward pass
about to backward
0.2982831075787544
here
forward pass
about to backward
0.3399181254208088
here
forward pass
about to backward
0.4468143843114376
here
forward pass
about to backward
0.5337329991161823
here
forward pass
about to backward
0.6367119364440441
here
forward pass
about to backward
0.7376743815839291
here
forward pass
about to backward
0.8436604924499989
here
forward pass
about to backward
0.9262244664132595
here
forward pass
about to backward
0.9744952023029327
here
forward pass
about to backward
1.048001043498516
here
forward pass
about to backward
1.1290441527962685
here
forward pass
about to backward
1.2275926992297173
here
forward pass
about to backward
1.273018229752779
here
forward pass
about to backward
1.3428944684565067
here
forward pass
about to backward
1.4152229093015194
here
forward pass
about to backward
1.48155153170228
here


In [34]:
model.eval()
nb_eval_steps, nb_eval_examples = 0, 0
predictions , true_labels = [], []

input_ids_list = []
print(len(test_dataloader))
count = 0
for batch in test_dataloader:
    print(count)
    count += 1
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # The model must not compute or store gradients
    with torch.no_grad():
        # Forward pass, calculate predictions.
        outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
    # Transfer logits and labels to CPU
    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    input_ids_list.extend(b_input_ids)
    
    # Calculate the accuracy for this batch of test sentences
    # eval_loss += outputs[0].mean().item()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)

pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
test_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
print("Test Accuracy: {}".format(accuracy_score(pred_tags, test_tags),digits=4))
print("Test classification report: {}".format(classification_report(pred_tags, test_tags,digits=4)))
print()

100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Test Accuracy: 0.7959074096169908


TypeError: ignored

In [35]:
len(pred_tags)

16762

In [36]:
len(test_tags)

16762

In [44]:
x = np.array(test_tags) == np.array(pred_tags)
np.sum(x) / len(test_tags)

0.7959074096169908

In [ ]:
tag_values = list(set(all_data["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
print(tag_values)

['O', 'B-MT', 'I-MT', 'PAD']


In [ ]:
MAX_LEN = 75
bs = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(list(all_data["Word"]), list(all_data["Tag"]))
]
tokenized_texts_and_labels

[(['h'], ['O']),
 (['w'], ['O']),
 (['c'], ['O']),
 (['a'], ['O']),
 (['r'], ['O']),
 (['f'], ['O']),
 (['a'], ['O']),
 (['i'], ['O']),
 (['a'], ['O']),
 (['h'], ['O']),
 (['m'], ['O']),
 (['s'], ['O']),
 (['b'], ['O']),
 (['j'], ['O']),
 (['t'], ['O']),
 (['f'], ['O']),
 (['a'], ['O']),
 (['t'], ['O']),
 (['u'], ['O']),
 (['o'], ['O']),
 (['u'], ['O']),
 (['i'], ['O']),
 (['d'], ['O']),
 (['a'], ['O']),
 (['h'], ['O']),
 (['e'], ['O']),
 (['m'], ['O']),
 (['a'], ['O']),
 (['i'], ['O']),
 (['t'], ['O']),
 (['v'], ['O']),
 (['m'], ['O']),
 (['p'], ['O']),
 (['o'], ['O']),
 (['t'], ['O']),
 (['a'], ['O']),
 (['t', 'h', 'y', 'r'], ['B', '-', 'M', 'T']),
 (['a', 's', 's', 'o'], ['I', '-', 'M', 'T']),
 (['i', 'n'], ['I', '-']),
 (['a', 'n', 'd'], ['I', '-', 'M']),
 (['t', 'h', 'e'], ['I', '-', 'M']),
 (['g', 'e', 'r', 'a'], ['I', '-', 'M', 'T']),
 (['a', 'u', 'r', 'b'], ['I', '-', 'M', 'T']),
 (['a', 'w', 'a', 'r'], ['I', '-', 'M', 'T']),
 (['f', 'r', 'o', 'm'], ['I', '-', 'M', 'T']),
 (['t

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
# tokenized_texts
labels

[['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B', '-', 'M', 'T'],
 ['I', '-', 'M', 'T'],
 ['I', '-'],
 ['I', '-', 'M'],
 ['I', '-', 'M'],
 ['I', '-', 'M', 'T'],
 ['I', '-', 'M', 'T'],
 ['I', '-', 'M', 'T'],
 ['I', '-', 'M', 'T'],
 ['I', '-', 'M'],
 ['B', '-', 'M', 'T'],
 ['I', '-', 'M', 'T'],
 ['I', '-', 'M'],
 ['I', '-', 'M', 'T'],
 ['I', '-', 'M', 'T'],
 ['I', '-'],
 ['B', '-', 'M', 'T'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B', '-', 'M', 'T'],
 ['I', '-', 'M'],
 ['I', '-', 'M', 'T'],
 ['I', '-', 'M'],
 ['B', '-', 'M', 'T'],
 ['I', '-', 'M', 'T'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")
# input_ids

array([[177,   0,   0, ...,   0,   0,   0],
       [192,   0,   0, ...,   0,   0,   0],
       [172,   0,   0, ...,   0,   0,   0],
       ...,
       [172,   0,   0, ...,   0,   0,   0],
       [187,   0,   0, ...,   0,   0,   0],
       [119,   0,   0, ...,   0,   0,   0]])

In [ ]:
# print(tokenized_texts)
tags = pad_sequences([[tag2idx.get(''.join(lab))] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")


[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [2], [None], [None], [None], [2], [2], [2], [2], [None], [1], [2], [None], [2], [2], [None], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [None], [2], [None], [1], [2], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [2], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [None], [0], [0], [1], [2], [0], [0], [1], [2], [1], [0], [0], [0], [0], [1], [2], [0], [0], [0], [1], [None], [2], [0], [0], [0], [0], [0], [0], [1], [2], [1], [0], [0], [1], [2], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0],

TypeError: ignored